In [2]:
from vfb_connect.cross_server_tools import VfbConnect
from vfb_connect.neo.neo4j_tools import dict_cursor
import neuprint 
from neuprint import fetch_neurons, NeuronCriteria as NC
import json
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
##Adding load variables from .env file (a local .env file is required)
#load .env
from dotenv import load_dotenv
load_dotenv()
import os
#define variables from .env
neuPRINT_API_token = os.getenv('neuPRINT_API_token')

In [4]:
import neuprint

# see https://github.com/connectome-neuprint/neuprint-python

# login to neuprint with GitHub & get your token here: https://neuprint.janelia.org/account - then past below
token = neuPRINT_API_token
client = neuprint.Client('https://neuprint.janelia.org', dataset='hemibrain:v1.1', token=token) # not 1.0.1?

In [5]:
from neuprint import Client

In [6]:
client.fetch_version()
import pandas as pd

In [7]:
from neuprint.queries import fetch_roi_hierarchy

roi_tree_text = fetch_roi_hierarchy(False, True, 'text')

print(roi_tree_text)

hemibrain
 +-- AL(L)*
 +-- AL(R)*
 +-- AOT(R)
 +-- CX
 |   +-- AB(L)*
 |   +-- AB(R)*
 |   +-- EB*
 |   +-- FB*
 |   +-- NO*
 |   +-- PB*
 +-- GC
 +-- GF(R)
 +-- GNG*
 +-- INP
 |   +-- ATL(L)*
 |   +-- ATL(R)*
 |   +-- CRE(L)*
 |   +-- CRE(R)*
 |   +-- IB*
 |   +-- ICL(L)*
 |   +-- ICL(R)*
 |   +-- SCL(L)*
 |   +-- SCL(R)*
 +-- LH(R)*
 +-- LX(L)
 |   +-- BU(L)*
 |   +-- LAL(L)*
 +-- LX(R)
 |   +-- BU(R)*
 |   +-- LAL(R)*
 +-- MB(+ACA)(R)
 |   +-- MB(R)
 |   |   +-- CA(R)*
 |   |   +-- PED(R)*
 |   |   +-- a'L(R)*
 |   |   +-- aL(R)*
 |   |   +-- b'L(R)*
 |   |   +-- bL(R)*
 |   |   +-- gL(R)*
 |   +-- dACA(R)
 |   +-- lACA(R)
 |   +-- vACA(R)
 +-- MB(L)
 |   +-- CA(L)*
 |   +-- a'L(L)*
 |   +-- aL(L)*
 |   +-- b'L(L)*
 |   +-- bL(L)*
 |   +-- gL(L)*
 +-- OL(R)
 |   +-- AME(R)*
 |   +-- LO(R)*
 |   +-- LOP(R)*
 |   +-- ME(R)*
 +-- PENP
 |   +-- CAN(R)*
 |   +-- FLA(R)*
 |   +-- PRW*
 |   +-- SAD*
 +-- POC
 +-- SNP(L)
 |   +-- SIP(L)*
 |   +-- SMP(L)*
 +-- SNP(R)
 |   +-- SIP(R)*
 |   +-

In [8]:
neur_ids = client.fetch_custom("""MATCH (n:hemibrain_Neuron) WHERE exists(n.type) RETURN n.bodyId as ID""")
neur_ids

,ID
0,1324365879
1,297908801
2,297230760
3,541697718
4,5813078603
...,...
22588,2101391269
22589,5813055448
22590,5813057864
22591,5813027266


In [305]:
from neuprint import fetch_adjacencies, NeuronCriteria, fetch_simple_connections

In [ ]:
###simple ver for fetching connectivity from neuprint
#fails if there is an error at any point so not useable currently
#add_neur_raw=fetch_simple_connections(list(neur_ids['ID']), list(neur_ids['ID']), client=client)

In [342]:
###fetches connectivity from neuprint and works through errors with single neurons.
#takes ~8 hours to run

all_neur=[]

for i in range(len(neur_ids)):
    try:
        #Fetch all downstream connections FROM a set of neurons
        neuron_df, conn_df = fetch_adjacencies(neur_ids['ID'][i], None) #don't need this if use None in fetch_simple
        #fetch simple connectitions between the upstream neuron and each downstream from conn_df
        per_neur=fetch_simple_connections(neur_ids['ID'][i], list(conn_df['bodyId_post']), client=client)
        all_neur.append(per_neur)
    except:
        continue

#concat all rows of all rois of all neurons        
all_neur_raw=pd.concat(all_neur)

#save to tsv as backup
all_neur_raw.to_csv("neuron_neuron_raw.tsv", sep="\t", index=False)

all_neur_raw

,bodyId_pre,bodyId_post,weight,type_pre,type_post,instance_pre,instance_post,conn_roiInfo
0,1324365879,5813057579,247,5-HTPLP01,AVLP538,5-HTPLP01_R,AVLP538_R,"{'VLNP(R)': {'pre': 231, 'post': 231}, 'AVLP(R..."
1,1324365879,1540241926,225,5-HTPLP01,AVLP500,5-HTPLP01_R,AVLP500_R,"{'VLNP(R)': {'pre': 208, 'post': 208}, 'AVLP(R..."
2,1324365879,1449486007,166,5-HTPLP01,AVLP537,5-HTPLP01_R,AVLP537_R,"{'VLNP(R)': {'pre': 166, 'post': 166}, 'AVLP(R..."
3,1324365879,1435791739,154,5-HTPLP01,PVLP132,5-HTPLP01_R,PVLP132_R,"{'VLNP(R)': {'pre': 154, 'post': 154}, 'PVLP(R..."
4,1324365879,1480525262,133,5-HTPLP01,AVLP536,5-HTPLP01_R,AVLP536_R,"{'VLNP(R)': {'pre': 133, 'post': 133}, 'AVLP(R..."
...,...,...,...,...,...,...,...,...
267,1570922129,5813075517,1,vpoEN,AVLP058,vpoEN_R,AVLP058_R,"{'VLNP(R)': {'pre': 1, 'post': 1}, 'AVLP(R)': ..."
268,1570922129,5813075524,1,vpoEN,AVLP005,vpoEN_R,AVLP005_R,"{'VLNP(R)': {'pre': 1, 'post': 1}, 'AVLP(R)': ..."
269,1570922129,5813078004,1,vpoEN,WED106,vpoEN_R,WED106_R,"{'VLNP(R)': {'pre': 1, 'post': 1}, 'PVLP(R)': ..."
270,1570922129,5901218599,1,vpoEN,None,vpoEN_R,None,"{'VLNP(R)': {'pre': 1, 'post': 1}, 'AVLP(R)': ..."


In [343]:
###column renames and drop unneeded columns
#copy simple_df
all_neur=all_neur_raw
#rename 'bodyId_pre', 'bodyId_post' and 'conn_roiInfo' columns to xrefs and 'weight_per_roi'
all_neur=all_neur.rename(columns={'bodyId_pre': 'subject_external_id', 'bodyId_post': 'object_external_id', 'conn_roiInfo': 'weight_per_roi'})
#add 'subject_external_db', 'object_external_db' and 'relation' columns
all_neur['subject_external_db']='neuprint_JRC_Hemibrain_1point1'
all_neur['object_external_db']='neuprint_JRC_Hemibrain_1point1'
all_neur['relation']='synapsed_to'
#drop type_pre/post and instance_pre/post
all_neur=all_neur.drop(['type_pre', 'type_post', 'instance_pre', 'instance_post'], axis=1)

###remove when doing roi loading
all_neur=all_neur.drop(['weight_per_roi'], axis=1)

all_neur

,subject_external_id,object_external_id,weight,subject_external_db,object_external_db,relation
0,1324365879,5813057579,247,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
1,1324365879,1540241926,225,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
2,1324365879,1449486007,166,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
3,1324365879,1435791739,154,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
4,1324365879,1480525262,133,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
...,...,...,...,...,...,...
267,1570922129,5813075517,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
268,1570922129,5813075524,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
269,1570922129,5813078004,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
270,1570922129,5901218599,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to


In [280]:
###still needs fixing, if there is no pre it can't work

#edit roi json
#per_roi_list=[]

#for connection in range(len(all_neur['weight_per_roi'])):
#    roi_json=all_neur['weight_per_roi'][connection]
#    for roino in range(len(roi_json)-1):
        #define roi and n for json row
#        roi=list(roi_json)[roino]
#        n=roi_json[list(roi_json)[roino]]['pre']
        #create per roi json ??need to make this a loop to cover all elements of original json   
#        per_roi = json.dumps({'roi': roi, 'weight': n})
#        per_roi_list.append(per_roi)
#    all_neur['weight_per_roi'][i]='|'.join(per_roi_list)
    
#all_neur

In [344]:
###remove downstream neurons which were not previously loaded in the db (no types)
#required when downstream neurons are not specified during fetching connectivity
all_rel_neur = all_neur[all_neur['object_external_id'].isin(neur_ids['ID'])]
all_rel_neur

,subject_external_id,object_external_id,weight,subject_external_db,object_external_db,relation
0,1324365879,5813057579,247,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
1,1324365879,1540241926,225,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
2,1324365879,1449486007,166,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
3,1324365879,1435791739,154,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
4,1324365879,1480525262,133,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
...,...,...,...,...,...,...
266,1570922129,5813070171,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
267,1570922129,5813075517,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
268,1570922129,5813075524,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to
269,1570922129,5813078004,1,neuprint_JRC_Hemibrain_1point1,neuprint_JRC_Hemibrain_1point1,synapsed_to


In [345]:
###write tsv for loading
from datetime import date
all_rel_neur.to_csv('newmeta_' + 'Xu2020NeuronsV1point1' + '_' + date.today().strftime('%Y%m%d')[2:8] + '.tsv', sep = '\t', index = False)

In [346]:
###create yaml data and write file
#yaml data
import yaml
yaml_data = dict(
    DataSet='Xu2020NeuronsV1point1',
    Curator='adm71')
#write yaml file for each tsv half
with open(r'newmeta_' + 'Xu2020NeuronsV1point1' + '_' + date.today().strftime('%Y%m%d')[2:8] + '.yaml', 'w') as file:
    documents = yaml.dump(yaml_data, file)

In [ ]:
#Remaining todo after this load: map roi to VFB id

In [351]:
###List neurons which failed to load query
missing_neur = neur_ids[~neur_ids['ID'].isin(all_rel_neur['object_external_id'])]
missing_neur
#fetch_adjacencies, half give key error, rest work
#fetch_simple_connections

,ID
2648,1545158404
7973,1348498693
7982,1536399125
8012,5813068978
8024,1721313731
12848,1669018125
15087,1949359185
15126,1821744519
16103,266276935
16106,296617104


In [ ]:
###simple ver for fetching connectivity from neuprint
#add_neur_raw=fetch_simple_connections(list(missing_neur['ID']), list(neur_ids['ID']), client=client)

In [ ]:
#1545158404 no synapses, just a cell body and primary neurite; 5813068978-1 output synapse
#all others work?